In [11]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [12]:
from dotenv import load_dotenv

load_dotenv() #On lit l'API de OPENAI

True

In [13]:
import pandas as pd

content = pd.read_csv("Content/content_cleaned.csv")

In [14]:
content["tagged_text"].to_csv("Content/Text/tagged_text.txt", lineterminator="\n", index=False, header=False)
content["tagged_caption"].to_csv("Content/Text/tagged_caption.txt", lineterminator="\n", index=False, header=False)
#On prend "text" et "caption" et on le transforme en un fichier txt

In [15]:
raw_documents = TextLoader("Content/Text/tagged_caption.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1915, which is longer than the specified 1
Created a chunk of size 220, which is longer than the specified 1
Created a chunk of size 342, which is longer than the specified 1
Created a chunk of size 297, which is longer than the specified 1
Created a chunk of size 331, which is longer than the specified 1
Created a chunk of size 148, which is longer than the specified 1
Created a chunk of size 181, which is longer than the specified 1
Created a chunk of size 386, which is longer than the specified 1
Created a chunk of size 132, which is longer than the specified 1
Created a chunk of size 263, which is longer than the specified 1
Created a chunk of size 140, which is longer than the specified 1


In [16]:
db_content = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())

In [23]:
query = "A woman who broke my heart"
docs = db_content.similarity_search(query, k=10)
docs

[Document(id='f64cd1f5-2785-426c-962b-aa4a06ff5ad7', metadata={'source': 'Content/Text/tagged_caption.txt'}, page_content='a38f15ad-b8b5-4bb0-89a5-a8f44fd4d302: The woman that I would try  Is happy with a good guy  But I\'ve been hackin\' so much\n"48e1a3b8-bf01-413e-86de-6fa2010d4a71: Lucky that you stayed on  I need someone to put this weight on  Fuck, I\'m sorry"\n44e2dd66-85a4-41c3-ad2b-abe4d7484258: We live in a generation of not being in love and not being together  But we sure make it feel like we\'re together  \'Cause we\'re scared to see each other with somebody else\n"c4ce4c7a-0212-4749-ad65-91b69d7f661f: When a good thing goes bad, it\'s not the end of the world  It\'s just the end of a world that you had with one girl  And she\'s the reason it happened, but she\'s overreacting  And it\'s all because she don\'t want things to change"\n"0b7e90ae-628c-480a-8d26-2b5307be9ebc: The biggest hit and second single from Kanye’s third studio album, Graduation, “Stronger” samples Frenc

In [26]:

content[content["id"] == docs[0].page_content.split()[0].strip()] #L'on retrouve les elements important pour retrouver le contenu recommander
docs[0].page_content.split()[0].strip()

'a38f15ad-b8b5-4bb0-89a5-a8f44fd4d302:'

In [24]:
#On cree une fonction qui va automatiser nos 3 dernieres cellules
def retrieve_semantic_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
    recs = db_content.similarity_search(query, k=50)

    content_list = []
    for i in range(0, len(recs)):
        clean = recs[i].page_content.split()[0].strip()
        content_list += [clean]

    return content[content["id"].isin(content_list)]

In [25]:
retrieve_semantic_recommendations("A woman who broke my heart")

,id,user,text,image,caption,created_at,like,views,bookmarks,category,vote,tagged_text,tagged_caption
